# Advent of Code 2023 - Day 22: **Sand Slabs** 

In [581]:
import collections
import numpy as np

with open('input.txt', 'r') as f:
    data = f.read().splitlines()

In [582]:
class Brick:
    def __init__(self, idx, x1, y1, z1, x2, y2, z2):
        self.x1 = min(x1, x2)
        self.x2 = max(x1, x2)
        self.y1 = min(y1, y2)
        self.y2 = max(y1, y2)
        self.z1 = min(z1, z2)
        self.z2 = max(z1, z2)
        self.idx = idx
        
    def __repr__(self):
        return f'B{self.idx}({self.x1},{self.y1},{self.z1})-({self.x2},{self.y2},{self.z2})'
    
    def __lt__(self, other):
        return self.z1 < other.z1
    
    def __eq__(self, other):
        return self.idx == other.idx
    
    def horizontal_intersect(self, other):
        return (self.x1 <= other.x2 and self.x2 >= other.x1 and
                self.y1 <= other.y2 and self.y2 >= other.y1)
        
    def __hash__(self) -> int:
        return hash(self.idx)

In [583]:
bricks = {}
z_map = collections.defaultdict(list) # only lower end of brick
brickmap = {} # all bricks and all coordinates

for i, line in enumerate(data):
    t = list(map(lambda x : list(map(int, x.split(','))) , line.split('~')))
    [[x1, y1, z1], [x2, y2, z2]] = t
    b = Brick(i, x1, y1, z1, x2, y2, z2)
    bricks[i] = b
    z_map[z1].append(b)
    for x in range(b.x1, b.x2+1):
        for y in range(b.y1, b.y2+1):
            for z in range(b.z1, b.z2+1):
                brickmap[(x,y,z)] = b

In [584]:
# test if brick is supported; provide own id to ignore
def is_supported(b):
    for x in range(b.x1, b.x2+1):
        for y in range(b.y1, b.y2+1):
            if (x, y, b.z1-1) in brickmap or b.z1 <= 1:
                return True
    return False

def get_supporting_bricks(b):
    supporting = set()
    for x in range(b.x1, b.x2+1):
        for y in range(b.y1, b.y2+1):
            if (x,y,b.z1-1) in brickmap:
                supporting.add(brickmap[(x,y,b.z1-1)])
    return supporting

In [585]:
support_by_map = collections.defaultdict(set)
supports_map = {}

# let fall down
for height in sorted(z_map):
    bricks_at_height = list(z_map[height])
    for b in bricks_at_height:
        floating = True
        while floating:
            if is_supported(b):
                supporting_bricks = get_supporting_bricks(b)
                support_by_map[b.idx] = supporting_bricks
                for s in supporting_bricks:
                    if s.idx not in supports_map:
                        supports_map[s.idx] = set()
                    supports_map[s.idx].add(b)
                floating = False
            else:
                b.z1 -= 1
                b.z2 -= 1
                for x in range(b.x1, b.x2+1):
                    for y in range(b.y1, b.y2+1):
                        brickmap[(x,y,b.z1)] = b
                        brickmap.pop((x,y,b.z2+1))
                z_map[b.z1].append(b)
                z_map[b.z1+1].remove(b)

In [586]:
def is_desintegratable(b):
    for parent in z_map[b.z2+1]:
        if parent.horizontal_intersect(b):
            if support_by_map[parent.idx] == {b}:
                return False
    return True

In [587]:
desintegratable = set()

for b in bricks.values():
    if is_desintegratable(b):
        desintegratable.add(b)
        
len(desintegratable)

477

In [588]:
def get_fall_count(b):
    queue = collections.deque([b])
    falling = set(queue)
    
    while queue:
        current = queue.popleft()
        if current.idx not in supports_map:
            continue
        for parent in supports_map[current.idx]:
            if len(support_by_map[parent.idx] - falling) == 0:
                queue.append(parent)
                falling.add(parent)
    return len(falling) - 1

In [589]:
total = 0
for height in z_map:
    bricks_at_height = list(z_map[height])
    for b in bricks_at_height:
        if b not in desintegratable:
            total += get_fall_count(b)
                
total

61555